# Organize ML experiments

## Introduction

This guide will show you how to:

- Keep track of code, data, environment and parameters
- Log results like evaluation metrics and model files
- Find experiments in the experiment dashboard with tags
- Organize experiments in a dashboard view and save it for later

## Setup

Install dependencies

In [1]:
pip install scikit-learn==0.23.1 joblib==0.15.1 neptune-client==0.4.119

Note: you may need to restart the kernel to use updated packages.


In [2]:
from IPython.display import HTML

## Step 1: Create a basic training script

In [3]:
from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from joblib import dump

data = load_wine()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target,
                                                    test_size=0.4, random_state=1234)

params = {'n_estimators': 10,
          'max_depth': 3,
          'min_samples_leaf': 1,
          'min_samples_split': 2,
          'max_features': 3,
          }

clf = RandomForestClassifier(**params)

clf.fit(X_train, y_train)
y_train_pred = clf.predict_proba(X_train)
y_test_pred = clf.predict_proba(X_test)

train_f1 = f1_score(y_train, y_train_pred.argmax(axis=1), average='macro')
test_f1 = f1_score(y_test, y_test_pred.argmax(axis=1), average='macro')
print(f'Train f1:{train_f1} | Test f1:{test_f1}')

Train f1:1.0 | Test f1:0.9737654320987654


## Step 2: Initialize Neptune

Connects your script to Neptune application. 

In [4]:
import neptune

neptune.init(
    api_token="ANONYMOUS",
    project_qualified_name="shared/onboarding"
)

Project(shared/onboarding)

You tell Neptune: 

* **who you are**: your Neptune API token `api_token` 
* **where you want to send your data**: your Neptune project `project_qualified_name`.

---

**Note:** 


Instead of logging data to the public project 'shared/onboarding' as an anonymous user 'neptuner' you can log it to your own project.

To do that:

1. Get your Neptune API token

![image](https://neptune.ai/wp-content/uploads/get_token.gif)

2. Pass the token to ``api_token`` argument of ``neptune.init()`` method: ``api_token=YOUR_API_TOKEN``
3. Pass your username to the ``project_qualified_name`` argument of the ``neptune.init()`` method: ``project_qualified_name='YOUR_USERNAME/sandbox``. Keep `/sandbox` at the end, the `sandbox` project that was automatically created for you.

For example:

```python
neptune.init(project_qualified_name='funky_steve/sandbox', 
             api_token='eyJhcGlfYW908fsdf23f940jiri0bn3085gh03riv03irn',
            )
```

---

## Step 3: Create an experiment and save parameters

In [5]:
neptune.create_experiment(name='great-idea', params=params)

NVMLError: NVML Shared Library Not Found - GPU usage metrics may not be reported.


https://ui.neptune.ai/shared/onboarding/e/ON-266


Experiment(ON-266)

This opens a new "experiment" namespace in Neptune to which you can log various objects.

Click on the link above to open this experiment in Neptune.

For now it is empty but keep the tab with experiment open to see what happens next. 

## Step 4. Add tags to organize things

Pass a list of strings to the ``.append_tag`` method of the experiment object.

In [6]:
neptune.append_tag(['experiment-organization', 'me'])

## Step 5. Add logging of train and evaluation metrics

In [7]:
neptune.log_metric('train_f1', train_f1)
neptune.log_metric('test_f1', test_f1)

Log all the metrics you care about with ``.log_metric`` method. There could be as many as you like.
The first argument is the name of the metric, the second it's value.

---

**Note:**

    You can log multiple values to the same metric. When you do that a chart will be created automatically.
    
---

## Step 6. Run a few experiments with different parameters

Let's run some experiments with different model configuration.

1. Change parameters in the ``params`` dictionary of the **Step 1: Create a basic training script**

```python

    params = {'n_estimators': 10,
              'max_depth': 3,
              'min_samples_leaf': 1,
              'min_samples_split': 2,
              'max_features': 3,
              }
``` 

2. Run all the cells, log things to Neptune.

3. Stop Neptune experiment. 

---

**Note:**
   
When you track experiments with Neptune in Jupyter notebooks you need to explicitly stop the experiment by running `neptune.stop()`.

If you are running Neptune in regular `.py` scripts it will stop automatically when your code stops running.

---

In [8]:
neptune.stop()

## Step 7. Go to Neptune UI

Click on one of the links created when you run the script or go directly to the app.

If you created your own project in Neptune you can also go to projects tab and find it.

In [9]:
HTML('<iframe width="720" height="420" src="https://www.youtube.com/embed/rEC-sxhP72w" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

/home/jakub/miniconda3/envs/py_37/lib/python3.7/site-packages/IPython/core/display.py:717: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


If you are logging things to the public project ``shared/onboarding`` you can just [follow this link](https://ui.neptune.ai/o/shared/org/onboarding/e/ON-261).

## Step 8. See that everything got logged

Go to one of the experiments you ran and see that you logged things correctly:

- click on the experiment link or one of the rows in the experiment table in the UI
- Go to ``Logs`` section to see your metrics
- Go to ``Source code`` to see that your code was logged
- Go to ``Artifacts`` to see that the model was saved

In [10]:
HTML('<iframe width="720" height="420" src="https://www.youtube.com/embed/WpAq7Kj88ec" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

## Step 9. Filter experiments by tag

Go to the experiments space and:

1. Click on the ``go to simple search``
2. In the ``tags`` type ``experiment-organization`` to find it (or other tag you added to your experiment).
3. Select the tag.

Neptune should filter all those experiments for you.

In [11]:
HTML('<iframe width="720" height="420" src="https://www.youtube.com/embed/ppPOtU_lNkk" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

## Step 10. Choose parameter and metric columns you want to see

Use the ``Manage columns`` button to choose the columns for the experiment table:

- Click on ``Manage columns``
- Go to the ``Numeric logs`` and ``Text parameters`` or type a name of your metric or parameter to find it.
- Add ``test_f1`` metric and the parameters you tweaked (in my case ``max_depth``, ``max_features``, ``min_samples_leaf``, and ``n_estimators``).

In [12]:
HTML('<iframe width="720" height="420" src="https://www.youtube.com/embed/gvlIXa25-Bc" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

---

**tip:**

    You can also use the suggested columns which shows you the columns with values that differ between selected experiments.

    Just click on the ``+`` to add it to your experiment table.
    
---

## Step 13. Save the view of experiment table

You can save the current view of experiment table for later:

- Click on the ``Save as new``

Both the columns and the filtering on rows will be saved as view.

In [13]:
HTML('<iframe width="720" height="420" src="https://www.youtube.com/embed/iTgjtYBWqko" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

---

**tip:**

    Create and save multiple views of the experiment table for different use cases or experiment groups.

---